# Estimate Continuous-time outcome measures
This `R` notebook performs hazard rate regressions to estimate the predicted outcomes of the game that are measured continuously as users take actions.

In [1]:
library(plyr)
library(survival)

In [2]:
working_dir = "../results-anonymized/pilot/"
filenames <- Sys.glob(paste(working_dir,"block_*_hazards.csv", sep=""))
filenames

[1] "../results-anonymized/pilot/block_20200505_pilot_hazards.csv"
[2] "../results-anonymized/pilot/block_20200506_pilot_hazards.csv"
[3] "../results-anonymized/pilot/block_20200507_pilot_hazards.csv"
[4] "../results-anonymized/pilot/block_20200624_pilot_hazards.csv"
[5] "../results-anonymized/pilot/block_20200626_pilot_hazards.csv"

In [79]:
# load the total dataset
df <- ldply(lapply(filenames, read.csv, header=TRUE, stringsAsFactors=TRUE), rbind)
names(df)

[1] "X"                            "start"                       
 [3] "exposure_id"                  "player_id"                   
 [5] "game_id"                      "is_treatment_condition"      
 [7] "is_spoke"                     "is_link_or_spur"             
 [9] "is_prompt"                    "is_in_leads"                 
[11] "is_in_deads"                  "n_exposures"                 
[13] "n_existing_leads"             "in_startup"                  
[15] "in_peak"                      "in_tail"                     
[17] "in_close"                     "n_rim_connections"           
[19] "n_triangle_paths"             "n_edges_shared_with_exposers"
[21] "is_link"                      "is_spur"                     
[23] "is_exposed_t0"                "is_held_t0"                  
[25] "stop"                         "adopt_event"                 
[27] "forget_event"                 "is_caveman_game"

In [5]:
# create subset that only includes clues that are candidates for adoption
# also, only look at spoke clues which are the same across conditions
# and ignore the clues an individual is exposed to at t0, because their neighbors didn't choose to adopt them
adopt_ht = subset(df, is_in_leads == 0 & n_exposures > 0 & is_spoke == 1 & is_exposed_t0==0)
dim(adopt_ht)

[1] 24990    28

In [70]:
# create interaction terms with treatment
adopt_ht$t_n_exposures = adopt_ht$n_exposures * adopt_ht$is_treatment_condition
adopt_ht$t_n_rim_connections = adopt_ht$n_rim_connections * adopt_ht$is_treatment_condition
adopt_ht$t_is_in_deads = adopt_ht$is_in_deads * adopt_ht$is_treatment_condition

adopt_ht$n_shared_edges <- adopt_ht$n_edges_shared_with_exposers #renaming, should be able to remove this

adopt_ht$t_n_shared_edges <- adopt_ht$n_shared_edges * adopt_ht$is_treatment_condition
tail(adopt_ht)

,X,start,exposure_id,player_id,game_id,is_treatment_condition,is_spoke,is_link_or_spur,is_prompt,is_in_leads,...,adopt_event,forget_event,is_caveman_game,t_n_exposures,t_n_rim_connections,t_is_in_deads,triangle_paths_exist,t_n_edges_shared_with_exposers,n_shared_edges,t_n_shared_edges
312179,535643,93.859,zkxoJdJcrFgimKzdW_tclue_2_8,zkxoJdJcrFgimKzdW,bALDaKPEGyutrm8bn,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
312180,701003,121.056,zkxoJdJcrFgimKzdW_tclue_2_8,zkxoJdJcrFgimKzdW,bALDaKPEGyutrm8bn,1,1,0,0,0,...,0,0,0,1,0,0,0,1,1,1
312181,816443,139.615,zkxoJdJcrFgimKzdW_tclue_2_8,zkxoJdJcrFgimKzdW,bALDaKPEGyutrm8bn,1,1,0,0,0,...,1,0,0,1,0,0,0,1,1,1
312235,1783644,270.249,zkxoJdJcrFgimKzdW_tclue_2_9,zkxoJdJcrFgimKzdW,bALDaKPEGyutrm8bn,1,1,0,0,0,...,0,0,0,1,0,0,0,2,2,2
312236,2192364,326.087,zkxoJdJcrFgimKzdW_tclue_2_9,zkxoJdJcrFgimKzdW,bALDaKPEGyutrm8bn,1,1,0,0,0,...,0,0,0,1,0,0,0,2,2,2
312237,2257884,335.312,zkxoJdJcrFgimKzdW_tclue_2_9,zkxoJdJcrFgimKzdW,bALDaKPEGyutrm8bn,1,1,0,0,0,...,1,0,0,1,1,0,0,3,3,3


# Estimate effect of familiarity, supporting pathways on adoption

Use this regression in the manipulation check - if there is no effect of n_triangle_paths, then we expect the treatment and control conditions to be identical, as there isn't interdependence between clues other than through the familiarity mechanism, and so the spoke clues are independent of one another.

In [7]:
factors_fit <- coxph(
    Surv(time=start, time2=stop, event=adopt_event, type='counting') ~
        in_startup + in_peak + in_close + in_tail + # timing dummies 
        n_exposures + t_n_exposures + # social reinforcement
        n_triangle_paths +  # "logical" support
        n_rim_connections + t_n_rim_connections + # familiarity with concepts
        is_in_deads + t_is_in_deads + # prior rejection
        n_existing_leads +  # current state of notebook
        frailty.gaussian(player_id),  # random player effect (as players have different play speeds)
    data=adopt_ht,
)

factors_fit

Warning message in coxpenal.fit(X, Y, strats, offset, init = init, control, weights = weights, :
“Inner loop failed to coverge for iterations 2 4 5 6”

Call:
coxph(formula = Surv(time = start, time2 = stop, event = adopt_event, 
    type = "counting") ~ in_startup + in_peak + in_close + in_tail + 
    n_exposures + t_n_exposures + n_triangle_paths + n_rim_connections + 
    t_n_rim_connections + is_in_deads + t_is_in_deads + n_existing_leads + 
    frailty.gaussian(player_id), data = adopt_ht)

                               coef  se(coef)       se2     Chisq  DF       p
in_startup                           0.00e+00  0.00e+00             1        
in_peak                    2.58e+01  3.53e+01  3.53e+01  5.32e-01   1    0.47
in_close                             0.00e+00  0.00e+00             1        
in_tail                              0.00e+00  0.00e+00             1        
n_exposures                5.01e-01  6.75e-02  6.27e-02  5.50e+01   1 1.2e-13
t_n_exposures             -1.15e-01  8.15e-02  6.47e-02  2.01e+00   1    0.16
n_triangle_paths           5.79e-02  5.65e-02  5.48e-02  1.05e+00   1    0.31
n_rim_connections          4

In [78]:
factors_summary = summary(factors_fit)
factors_coefs = merge(factors_summary$conf.int, factors_summary$coefficients, by='row.names')
rownames(factors_coefs) <- factors_coefs$Row.names  # reset row names
factors_coefs$Row.names <- NULL

# add means
factors_coefs$means <- factors_fit$means[row.names(factors_coefs)]

# export data for plotting in python
write.csv(factors_coefs, paste(working_dir, "factors_cox_results.csv", sep=""))
factors_coefs

,exp(coef),exp(-coef),lower .95,upper .95,coef,se(coef),se2,Chisq,DF,p,means
in_close,NA,NA,NA,NA,NA,0.000000000,0.00000000,NA,1,NA,0.148979592
in_peak,1.570433e+11,6.367670e-12,1.310891e-19,1.881362e+41,25.77978753,35.336464165,35.33640104,0.5322461,1,4.656641e-01,0.269147659
in_startup,NA,NA,NA,NA,NA,0.000000000,0.00000000,NA,1,NA,0.003321329
in_tail,NA,NA,NA,NA,NA,0.000000000,0.00000000,NA,1,NA,0.578551421
is_in_deads,1.510180e-01,6.621729e+00,1.093119e-01,2.086363e-01,-1.89035648,0.164897810,0.16232950,131.4188441,1,2.005062e-30,0.177751100
n_existing_leads,1.072663e+00,9.322596e-01,1.054160e+00,1.091490e+00,0.07014400,0.008877428,0.00745667,62.4319207,1,2.758162e-15,11.129891957
n_exposures,1.649851e+00,6.061155e-01,1.445281e+00,1.883376e+00,0.50068475,0.067542571,0.06268737,54.9507401,1,1.235888e-13,1.237454982
n_rim_connections,1.592242e+00,6.280453e-01,1.499628e+00,1.690575e+00,0.46514305,0.030574996,0.02968823,231.4409809,1,2.891298e-52,0.859743898
n_triangle_paths,1.059642e+00,9.437146e-01,9.485427e-01,1.183755e+00,0.05793146,0.056511208,0.05483761,1.0508959,1,3.053008e-01,0.265666267
t_is_in_deads,6.990039e-01,1.430607e+00,4.227174e-01,1.155870e+00,-0.35809897,0.256613109,0.25276142,1.9473700,1,1.628705e-01,0.089555822


# Estimate effect of similarity to exposers

The theoretical argument is that logic and familiarity increase polarization by making individuals imitate their more similar alters. So, we swap out the logic and familiarity regressors with the similarity measure

In [71]:
similarity_fit <- coxph(
    Surv(time=start, time2=stop, event=adopt_event, type='counting') ~
        in_startup + in_peak + in_close + in_tail +  # timing dummies 
        n_exposures + t_n_exposures +  # social reinforcement
        n_shared_edges + t_n_shared_edges +  # similarity to exposers
        is_in_deads + t_is_in_deads +  # prior rejection
        n_existing_leads +  # current state of notebook
        frailty.gaussian(player_id),  # random player effect (as players have different play speeds)
    data=adopt_ht,
)


similarity_fit

Warning message in coxpenal.fit(X, Y, strats, offset, init = init, control, weights = weights, :
“Inner loop failed to coverge for iterations 2 3 4 5 6 7 8 9 10”Warning message in sqrt((diag(x$var2))[kk]):
“NaNs produced”Warning message in sqrt((diag(x$var2))[kk]):
“NaNs produced”Warning message in sqrt((diag(x$var2))[kk]):
“NaNs produced”Warning message in sqrt((diag(x$var2))[kk]):
“NaNs produced”Warning message in sqrt((diag(x$var2))[kk]):
“NaNs produced”

Call:
coxph(formula = Surv(time = start, time2 = stop, event = adopt_event, 
    type = "counting") ~ in_startup + in_peak + in_close + in_tail + 
    n_exposures + t_n_exposures + n_shared_edges + t_n_shared_edges + 
    is_in_deads + t_is_in_deads + n_existing_leads + frailty.gaussian(player_id), 
    data = adopt_ht)

                               coef  se(coef)       se2     Chisq  DF       p
in_startup                 226.7349   87.4886              6.7164   1 0.00955
in_peak                                0.0000    0.0000             1        
in_close                               0.0000    0.0000             1        
in_tail                                0.0000    0.0000             1        
n_exposures                  0.3634    0.1008    0.0557   13.0040   1 0.00031
t_n_exposures                0.2394    0.1182              4.1017   1 0.04284
n_shared_edges              -0.0236    0.0261              0.8175   1 0.36592
t_n_shared_edges             0.0131    0.0272        

In [77]:
similarity_summary = summary(similarity_fit)
similarity_coefs = merge(similarity_summary$conf.int, similarity_summary$coefficients, by='row.names')
rownames(similarity_coefs) <- similarity_coefs$Row.names  # reset row names
similarity_coefs$Row.names <- NULL

# add means
similarity_coefs$means <- similarity_fit$means[row.names(similarity_coefs)]

# export data for plotting in python
write.csv(similarity_coefs, paste(working_dir, "similarity_cox_results.csv", sep=""))
similarity_coefs

Warning message in sqrt((diag(object$var2))[kk]):
“NaNs produced”Warning message in sqrt((diag(object$var2))[kk]):
“NaNs produced”Warning message in sqrt((diag(object$var2))[kk]):
“NaNs produced”Warning message in sqrt((diag(object$var2))[kk]):
“NaNs produced”Warning message in sqrt((diag(object$var2))[kk]):
“NaNs produced”

,exp(coef),exp(-coef),lower .95,upper .95,coef,se(coef),se2,Chisq,DF,p,means
in_close,NA,NA,NA,NA,NA,0.00000000,0.00000000,NA,1,NA,0.148979592
in_peak,NA,NA,NA,NA,NA,0.00000000,0.00000000,NA,1,NA,0.269147659
in_startup,2.949210e+98,3.390739e-99,9.984178e+23,8.711621e+172,226.73487633,87.48855536,NaN,6.7163631,1,9.553224e-03,0.003321329
in_tail,NA,NA,NA,NA,NA,0.00000000,0.00000000,NA,1,NA,0.578551421
is_in_deads,4.053821e-01,2.466808e+00,3.073351e-01,5.347084e-01,-0.90292519,0.14127376,0.09209751,40.8489157,1,1.644645e-10,0.177751100
n_existing_leads,1.095022e+00,9.132233e-01,1.069111e+00,1.121562e+00,0.09077485,0.01221849,NaN,55.1944793,1,1.091749e-13,11.129891957
n_exposures,1.438188e+00,6.953193e-01,1.180432e+00,1.752228e+00,0.36338418,0.10076914,0.05569089,13.0040005,1,3.108262e-04,1.237454982
n_shared_edges,9.766902e-01,1.023866e+00,9.280093e-01,1.027925e+00,-0.02358575,0.02608611,NaN,0.8174870,1,3.659159e-01,4.643657463
t_is_in_deads,3.080330e-01,3.246406e+00,1.927032e-01,4.923858e-01,-1.17754839,0.23931851,0.19506646,24.2105665,1,8.635719e-07,0.089555822
t_n_exposures,1.270506e+00,7.870879e-01,1.007749e+00,1.601774e+00,0.23941533,0.11821477,NaN,4.1016674,1,4.284094e-02,0.591796719
